In [1]:
from urllib import request
import lxml.html
#import sys
#!{sys.executable} -m pip install readability-lxml
#from readability import Document

In [2]:
url = "https://www.gismeteo.ru/weather-vladivostok-4877/10-days/"
output = request.urlopen(url).read().decode("utf-8") 

In [4]:
html = lxml.html.fromstring(output)

In [41]:
temperatures = html.find_class('value')

In [42]:
for i in range(10):
    print(temperatures[i][0].text,elems[i][1].text )

−12 −17
−11 −16
−11 −17
−4 −10
−3 −7
−9 −12
−7 −13
−6 −12
−3 −11
−7 −12


In [83]:
winds = html.find_class('w_wind')

In [86]:
winds[5][0].text

In [87]:
for i in range(30):
    print(winds[i][0].text)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


## Я хз почему это не работает. Должно выдавать ветер тупой, но не выдает